# An alternative method to estimate the traffic
Given the inflow and the outflow, given the data from OD, here we estimate the starting and the stopping based on these assumptions:
- ~~each trip lasts for `delta=20` minutes~~

































































































































































































































































































- the proportion of inflow stopping outside, `alpha`, is constant during the day
- the proportion of starting stopping outside, `beta`, is constant during the day

Note that all the analysis is performed considering `[inflow]=[outflow]=vehicles/5min`. When/if changing the flows, please check that the measurement unit is coherent and, if not, perform a proper rescaling

For a deeper explaination see the documentation (overleaf)

## Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Load input data

In [ ]:
# Inflow from gate data -> copied from traffic_model.ipynb
# FIXME: read from data lake
in_flow = np.array([ 
    221.25 , 208.70090169, 196.59507377, 184.93251626,173.71322915, 162.93721244, 152.60446612, 142.71499021,
    133.2687847 , 124.26584959, 115.70618488, 107.58979057,99.91666667, 92.68681316, 85.90023005, 79.55691735,
    73.65687504, 68.20010313, 63.18660163, 58.57940003,54.34152786, 50.4729851 , 46.97377176, 43.84388784,
    41.08333333, 38.69210825, 36.67021258, 35.01764633,33.73440951, 32.82050209, 32.2759241 , 31.97296144,
    31.78390001, 31.70873982, 31.74748086, 31.90012315,32.16666667, 32.54711142, 33.04145742, 33.64970465,
    34.37185312, 35.20790282, 36.15785376, 37.26801653,38.58470172, 40.10790933, 41.83763935, 43.7738918 ,
    45.91666667, 48.26596395, 50.82178366, 53.58412578,56.55299032, 59.72837728, 63.11028667, 67.02310605,
    71.79122301, 77.41463755, 83.89334968, 91.22735938,99.41666667, 108.46127153, 118.36117398, 129.11637401,
    140.72687162, 153.19266681, 166.51375958, 181.49843978,198.95499726, 218.88343202, 241.28374407, 266.15593339,
    293.5 , 323.31594389, 355.60376506, 390.36346351,427.59503924, 467.29849225, 509.47382255, 552.31727381,
    594.02508973, 634.59727031, 674.03381555, 712.33472545,749.5 , 785.52963921, 820.42364308, 854.18201161,886.80474479, 918.29184263, 948.64330513, 976.28773216,
    999.65372359, 1018.74127943, 1033.55039966, 1044.0810843 ,1050.33333333, 1052.30714677, 1050.00252461, 1043.41946686,
    1032.5579735 , 1017.41804455, 997.99967999, 977.02114803,957.20071686, 938.53838647, 921.03415686, 904.68802804,
    889.5 , 875.47007275, 862.59824628, 850.88452059,840.32889569, 830.93137158, 822.69194825, 815.37241668,
    808.73456786, 802.77840178, 797.50391844, 792.91111785,
    789. , 785.77056489, 783.22281253, 781.35674292,780.17235604, 779.66965191, 779.84863052, 780.46888892,
    781.29002415, 782.3120362 , 783.53492508, 784.9586908 ,786.58333333, 788.4088527 , 790.43524889, 792.66252192,
    795.09067177, 797.71969844, 800.54960195, 803.02721276,804.59936133, 805.26604768, 805.02727179, 803.88303368,
    801.83333333, 798.87817076, 795.01754596, 790.25145892,784.57990966, 778.00289817, 770.52042444, 763.0159827 ,
    756.37306713, 750.59167774, 745.67181454, 741.61347751,738.41666667, 736.081382 , 734.60762352, 733.99539122,
    734.24468509, 735.35550515, 737.32785139, 739.61300218,741.6622359 , 743.47555253, 745.0529521 , 746.39443459,
    747.5 , 748.36964834, 749.0033796 , 749.40119378,749.56309089, 749.48907093, 749.17913389, 749.28980052,
    750.47759157, 752.74250705, 756.08454694, 760.50371126,766. , 772.57341316, 780.22395074, 788.95161274,
    798.75639917, 809.63831001, 821.59734528, 834.56717618,848.48147392, 863.34023851, 879.14346994, 895.89116821,
    913.58333333, 932.2199653 , 951.8010641 , 972.32662975,993.79666225, 1016.21116159, 1039.57012777, 1062.26316093,
    1082.67986119, 1100.82022857, 1116.68426305, 1130.27196464,1141.58333333, 1150.61836914, 1157.37707205, 1161.85944207,
    1164.06547919, 1163.99518343, 1161.64855477, 1157.67561751,1152.72639595, 1146.80089008, 1139.89909992, 1132.02102544,
    1123.16666667, 1113.33602359, 1102.5290962 , 1090.74588451,1077.98638852, 1064.25060823, 1049.53854363, 1034.03498586,
    1017.92472607, 1001.20776426, 983.88410042, 965.95373456,947.41666667, 928.27289675, 908.52242481, 888.16525085,
    867.20137486, 845.63079684, 823.4535168 , 800.82280064,777.89191427, 754.66085769, 731.12963089, 707.29823388,
    683.16666667, 658.73492924, 634.0030216 , 608.97094374,583.63869568, 558.0062774 , 532.07368891, 506.86154363,
    483.39045498, 461.66042296, 441.67144757, 423.4235288 ,406.91666667, 392.15086116, 379.12611228, 367.84242003,
    358.29978441, 350.49820541, 344.43768305, 339.53803014,335.21905953, 331.48077121, 328.32316518, 325.74624144,
    323.75 , 322.33444085, 321.49956399, 321.24536942,321.57185714, 322.47902716, 323.96687946, 325.32925698,
    325.86000264, 325.55911644, 324.42659838, 322.46244845,319.66666667, 316.03925302, 311.58020751, 306.28953013,
    300.1672209 , 293.2132798 , 285.42770685, 276.81050203,267.36166534, 257.0811968 , 245.9690964 , 234.02536413
])


In [ ]:
# outflow from spira_outflow_estimation -> copied from traffic_model.ipynb
# FIXME: read from data lake
out_flow = np.array([
       244.77861109, 232.36180063, 220.31010111, 208.62351253, 197.30203489,
       186.34566818,175.75441241,165.52826759,155.6672337,146.17131075,
       137.04049873,128.27479766,119.87420753,111.61712941,103.51764292,
       95.421859,88.05247332,81.53910039,75.01201861,69.22820323,
       64.40462434,60.34125981,56.69849334,52.96060926,49.75703412,
       47.06492426,44.23577288,41.71759661,40.01412602,38.18565376,
       36.52013182,35.28771248,34.09765375,33.32988157,32.83798486,
       32.69738489,32.13946997,31.79701693,31.23879412,30.93867473,
       30.5640021,30.59874219,30.79466099,31.03077592,31.93174691,
       33.29475176,34.39689037,35.92930023,37.83949326,39.91341652,
       42.39509002,44.81192533,47.5384192,49.71481365,52.61063958,
       56.72776874,61.95912839,68.20500193,75.49621639,82.72885491,
       91.31445603,101.41624022,110.53830149,119.91744149,128.29367569,
       137.3044439,148.01729565,158.65164401,171.00279106,185.37319992,
       203.35224768,224.84527287,248.73161957,274.70356687,302.00168258,
       335.95429414,372.97087673,409.51906806,450.00389578,490.05345921,
       533.55773794,580.63860298,629.61163879,680.14375708,730.78596961,
       778.1512774,820.73965712,860.32924123,898.46887636,934.58231432,
       968.65499377,996.18424057,1019.61502728,1040.08299049,1056.24216434,
       1066.34255016,1072.03517003,1072.68889463,1069.75321886,1064.37262152,
       1051.10679859,1033.17600222,1014.04851418,991.16520903,971.78771141,
       953.03719135,933.41192626,914.56762274,894.11206199,874.82522003,
       857.50457749,843.84989723,835.4957252,826.66657803,818.58113138,
       811.60110191,806.1821823,803.40052708,801.78756715,800.75490253,
       801.31473649,801.32029098,802.22906712,803.17636515,803.99921557,
       805.19395131,804.89768189,805.30690617,806.84100202,808.84151383,
       809.84168243,810.26986571,811.32919139,813.19881944,816.58912385,
       817.35988121,818.86177139,817.43483681,817.40375931,818.64045015,
       819.53182184,818.67806316,816.35317422,814.22420199,810.25077482,
       807.38881518,804.60373757,799.56007132,793.77876372,788.46835342,
       782.04425342,778.3403178,775.3831065,771.05182847,767.30455689,
       763.97998856,763.95158912,762.29551127,763.78820844,763.63847724,
       766.79979479,768.82418072,770.86601848,773.32515585,775.6621998,
       774.83667276,775.69094159,777.49870728,777.32456026,779.10971842,
       779.23427254,779.73558778,779.86379242,780.17620562,779.86392324,
       781.69494431,783.86928861,785.25155125,788.55420018,792.55163616,
       800.03901809,807.98746048,821.88937843,832.85780734,843.35542339,
       852.14889085,859.59358912,869.91980805,884.08919856,899.83594137,
       916.83302306,933.9947141,950.69179834,971.31524852,993.94573372,
       1015.47510598,1032.70897486,1048.03731392,1061.755307,1074.2968419,
       1086.44808025,1095.5435283,1102.10935699,1106.75529337,1108.58548344,
       1107.93237746,1105.46997739,1097.41507381,1085.77455473,1072.92719682,
       1059.40019877,1048.20364213,1038.80820399,1027.66425604,1017.37605104,
       1007.45460334,999.65937805,995.26294604,990.2232526,982.94949868,
       974.99926005,968.15979324,962.74401047,957.73595049,953.46404853,
       948.01603374,939.3976317,930.77604676,921.51679499,912.10138653,
       901.73736461,889.31637584,872.95456852,856.33928322,837.50300708,
       816.65901769,796.61748972,774.80367426,752.24762291,726.29460789,
       700.11849818,673.04715023,646.07684746,618.49926521,590.97729494,
       563.71927366,536.58450601,511.68670661,487.57358623,464.10113576,
       440.90253384,422.12254553,405.35493876,390.39531929,377.22845451,
       365.62926244,354.67362878,345.62346399,338.27995421,331.63692211,
       327.50875561,323.53051575,320.03231947,317.1403173,315.1537921,
       314.08621384,313.11487363,313.14807188,313.26705928,313.95678114,
       315.76574196,317.94846131,318.64641421,319.70985092,320.88387816,
       322.21275853,320.59737932,318.4834929,315.87109928,312.76019845,
       309.15079042,305.04287519,300.43645276,295.33152313,289.72808629,
       283.62614225,277.02569101,269.92673256
])

In [ ]:
# Total inflow, from OD data (in passengers)
TI_tilda = 268358.572
TO_tilda = 266870.836
TInside_tilda = 342746.023

In [ ]:
# Total inflow, from OD data (in vehicles)
p2v = sum(in_flow) / TI_tilda

TI = TI_tilda * p2v; print(TI)
TO = TO_tilda * p2v; print(TO)
TInside = TInside_tilda * p2v; print(TInside)

In [ ]:
# Sum of traffic (from traffic model)
ST = 2480088.8075017566 # FIXME: from data lake

In [ ]:
t_unit = 5 # minutes
delta = 20 # minutes, average travel time
n_units = delta / t_unit #; print(n_units)
dur = int(24*60/t_unit) # number of time units in a day

## Compute starting and stopping with inverse formulas

In [ ]:
alpha_hat = TI / sum(in_flow)
beta_hat = (TInside) / (TInside + TO)

In [ ]:
lambdda = 1/(delta/t_unit)
tt = np.arange(dur+1)
p_t = (np.exp(-lambdda * tt) - np.exp(-lambdda * np.roll(tt, -1)))[:-1]
p_t

In [ ]:
t_limit = np.where(p_t < 0.001)[0][0] # 0.001 since a reduction of 0.1% is considered negligible

In [ ]:
p_t[t_limit:] = 0

In [ ]:
T_start = - (1-alpha_hat)*in_flow
for t in range(t_limit):
    T_start = T_start + np.roll(out_flow, shift=-t) * p_t[t]
T_start = T_start / (1 - beta_hat)
T_start

In [ ]:
c = alpha_hat*in_flow + beta_hat*T_start
Pmat = np.array([np.roll(p_t, shift=i) for i in range(dur)])

T_stop = np.linalg.solve(Pmat, c)

In [ ]:
plt.figure(figsize=(18,8))
plt.plot(T_start, label="Starting")
plt.plot(T_stop, label="Stopping")
plt.plot(in_flow, label="Inflow")
plt.plot(out_flow, label="Outflow")
plt.legend()

## Compute the traffic

In [ ]:
def solve(init, steps):
    series = init
    sequence = [series]
    for i in range(steps):
        mu = 1 + 3 * sum(series) / ST
        alfa = (mu-1)/mu
        series = init + np.roll(series,1) * alfa
        sequence += [series]
    return sequence

Traffics = solve(T_start+in_flow/5, 50)
for Traffic in Traffics:
    plt.plot(Traffic)